In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

data = pd.read_csv('D:/김홍민/김재윤 교수님 연구실/졸업 준비/clean_bitcoin_data_with_usd_krw_updated.csv')
# Convert 'Date' to datetime and set it as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
# 문자형인 종속 변수를 수치형으로 변환
data['rise_fall'] = data['rise_fall'].map({'rise': 1, 'fall': 0})
data.drop(['Unnamed: 0', 'change_percent'], axis=1, inplace=True) # delete adjusted close
data.rename(columns= {'USD_KRW_Exchange_Rate' : 'EXR'}, inplace= True)
# Data split based on the provided date threshold
idx = pd.Timestamp('2023-01-01')
train_data = data[data.index < idx]
test_data = data[data.index >= idx]

In [24]:

data = pd.read_csv('D:/김홍민/김재윤 교수님 연구실/졸업 준비/clean_bitcoin_data_with_usd_krw_updated.csv')

In [25]:
# Convert 'Date' to datetime and set it as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [26]:
# 문자형인 종속 변수를 수치형으로 변환
data['rise_fall'] = data['rise_fall'].map({'rise': 1, 'fall': 0})

In [27]:
data.drop(['Unnamed: 0', 'change_percent'], axis=1, inplace=True) # delete adjusted close
data.rename(columns= {'USD_KRW_Exchange_Rate' : 'EXR'}, inplace= True)

In [28]:
# Data split based on the provided date threshold
idx = pd.Timestamp('2023-01-01')
train_data = data[data.index < idx]
test_data = data[data.index >= idx]

In [29]:

# Feature와 Target 설정
target_column = 'rise_fall'
features = [col for col in data.columns if col not in ['Date', target_column]]

X_train = train_data[features]
y_train = train_data[target_column]

X_test = test_data[features]
y_test = test_data[target_column]

# 결측치 제거
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)



In [30]:

# Random Forest 모델 훈련
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [31]:
# 예측
y_pred = rf_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
classification = classification_report(y_test, y_pred)



In [32]:

# 결과 출력
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification)

Accuracy: 0.5070422535211268
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.82      0.62       313
           1       0.55      0.20      0.30       326

    accuracy                           0.51       639
   macro avg       0.52      0.51      0.46       639
weighted avg       0.52      0.51      0.45       639



In [33]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,

In [34]:
import pandas as pd
import os


result_folder = 'D:/김홍민/김재윤 교수님 연구실/졸업 준비/병아리'

# Function to process trading data
def process_trading_data(df):
    # 초기화
    buy_price = None
    sell_price = None
    holding = False
    df['Unrealized_profit'] = 0
    df['Cumulative_Profit'] = 0  # 누적 이익 초기화
    current_profit = 0
    initial_price = None
    for index, row in df.iterrows():
        action = row['action']
        profit = 0

        if action == 'Long':
            buy_price = row['Close']  # Long 시, 매수 가격을 설정
            holding = True
            if initial_price is None:  # 초기 가격 설정
                initial_price = buy_price
        elif action == 'Short':
            sell_price = row['Close']  # Short 시, 매도 가격을 설정
            holding = True
            if initial_price is None:  # 초기 가격 설정
                initial_price = sell_price
        elif action == 'Holding' and holding and buy_price is not None:
            df.at[index, 'Unrealized_profit'] = (row['Close'] - buy_price)
        elif action == 'Holding' and holding and sell_price is not None:
            df.at[index, 'Unrealized_profit'] = (sell_price - row['Close'])
        elif action == 'Exit' and holding and buy_price is not None:    
            df.at[index, 'Unrealized_profit'] = (row['Close'] - buy_price) - 0.01
            profit = df.at[index, 'Unrealized_profit']
            df.at[index, 'Cumulative_Profit'] = current_profit + profit  # 누적 이익 업데이트
            current_profit = df.at[index, 'Cumulative_Profit']
            buy_price = None
            holding = False
        elif action == 'Exit' and holding and sell_price is not None:
            df.at[index, 'Unrealized_profit'] = (sell_price - row['Close']) - 0.01
            profit = df.at[index, 'Unrealized_profit']
            df.at[index, 'Cumulative_Profit'] = current_profit + profit  # 누적 이익 업데이트
            current_profit = df.at[index, 'Cumulative_Profit']
            sell_price = None
            holding = False
        elif action == 'no action' and holding:
            pass

    # 수익률 계산 및 누적 수익 계산
    df['Cumulative_Return'] = (df['Cumulative_Profit'] / initial_price) * 100
    df['Unrealized_Return'] = (df['Unrealized_profit'] / initial_price) * 100

    # 누적 수익 + 미실현 수익률 컬럼 추가
    df['Unrealized_profit_plus_Cumulative_Profit'] = df['Cumulative_Profit'] + df['Unrealized_profit']
    df['Unrealized_profit_plus_Cumulative_Profit_Return'] = (df['Unrealized_profit_plus_Cumulative_Profit'] / initial_price) * 100
    df['Cum_portfolio'] = df['Cumulative_Profit'] + initial_price
    df['Unreal_Cum_portfolio'] = df['Unrealized_profit'] + initial_price

    # 매매 결과 총 정리
    df['action'] = df['action'].replace('no action', 0).replace('Long', 1).replace('Short', 2).replace('Exit', -1)

    # 거래 횟수 및 승률
    no_trade = len(df[df['action'] == -1]) - len(df[(df['action'] == -1) & (df['Unrealized_profit'] == 0)])
    winning_ratio = len(df[(df['action'] == -1) & (df['Unrealized_profit'] > 0)]) / no_trade if no_trade > 0 else 0

    profit_average = df[(df['action'] == -1) & (df['Unrealized_profit'] > 0)]['Unrealized_profit'].mean()
    loss_average = df[(df['action'] == -1) & (df['Unrealized_profit'] < 0)]['Unrealized_profit'].mean()

    # Payoff Ratio, Profit Factor
    payoff_ratio = profit_average / -loss_average if loss_average < 0 else 0
    profit_factor = -df[(df['action'] == -1) & (df['Unrealized_profit'] > 0)]['Unrealized_profit'].sum() / df[(df['action'] == -1) & (df['Unrealized_profit'] < 0)]['Unrealized_profit'].sum()

    # Maximum Drawdown (MDD) - 전체 데이터
    peak = df['Cum_portfolio'].cummax()  # 각 시점까지의 누적 최고치 추적
    drawdown = (peak - df['Cum_portfolio']) / peak  # 최고점에서 얼마만큼 떨어졌는지 계산
    max_drawdown = drawdown.max() * 100  # 최대 드로우다운(낙폭)을 계산

    peak_unreal = df['Unreal_Cum_portfolio'].cummax()  # 미실현 수익 포함한 포트폴리오의 누적 최고치 추적
    drawdown_unreal = (peak_unreal - df['Unreal_Cum_portfolio']) / peak_unreal  # 미실현 수익 포함한 드로우다운 계산
    portfolio_max_drawdown = drawdown_unreal.max() * 100  # 포트폴리오의 최대 드로우다운 계산
    
    # 누적 수익 및 손실
    max_cum_profit = df[df['Cumulative_Profit'] != 0]['Cumulative_Profit'].max()
    max_cum_loss = df[df['Cumulative_Profit'] != 0]['Cumulative_Profit'].min()
    last_cumulative = df['Cumulative_Profit'].iloc[-1]

    # 요약 데이터 프레임 생성
    summary_data = [[no_trade, winning_ratio, profit_average, loss_average, payoff_ratio, profit_factor, max_drawdown, portfolio_max_drawdown, max_cum_profit, max_cum_loss, last_cumulative]]
    summary_df = pd.DataFrame(summary_data, columns=['No_Trade', 'Winning_Ratio', 'Profit_Average', 'Loss_Average', 'Payoff_Ratio', 'Profit_Factor', 'Max_Drawdown', 'Portfolio_Max_Drawdown', 'Max_Cumulative_Profit', 'Max_Cumulative_Loss', 'Last_Cumulative'])
    
    return summary_df



In [ ]:
# LSTM 예측 결과를 기반으로 action 결정
actions = ['Holding']  # 첫 번째 데이터는 Holding으로 설정
previous_action = 'Holding'  # 첫 번째 행동은 보유로 시작


# 'predictions'는 시퀀스 데이터로 인해 test_data보다 길이가 짧을 수 있으므로 예측의 시작 지점부터 설정
for i in range(1, len(test_data)):
    if i < len(y_pred):  # 예측값이 있는 구간에 대해 액션 결정
        if y_pred[i] == 1 and previous_action != 'Long':  # 상승 예측 (Long)
            action = 'Long'
        elif y_pred[i] <= 0.5 and previous_action != 'Long':  # 하락 예측 (Short)
            action = 'Long'
        elif (y_pred[i] > 0.5 and previous_action == 'Exit') or (y_pred[i] <= 0.5 and previous_action == 'Long'):
            action = 'Exit'  # 반대 방향 예측 시 Exit   
        else:
            action = 'Holding'  # 변동이 적거나 동일한 예측일 경우
    else:
        action = 'Holding'  # 예측값이 없을 경우 Holding 유지

    actions.append(action)
    previous_action = action

    # Test data에 action 컬럼 추가
test_data['action'] = actions

    


C:\Users\johnkim\AppData\Local\Temp\ipykernel_4796\580132882.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['action'] = actions


In [36]:
# LSTM 모델의 예측을 기반으로 수익률을 평가하는 함수 호출
summary_df = process_trading_data(test_data)


C:\Users\johnkim\AppData\Local\Temp\ipykernel_4796\449873333.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unrealized_profit'] = 0
C:\Users\johnkim\AppData\Local\Temp\ipykernel_4796\449873333.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cumulative_Profit'] = 0  # 누적 이익 초기화
C:\Users\johnkim\AppData\Local\Temp\ipykernel_4796\449873333.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.109999999998544

In [37]:

# 성과 지표 출력
print(summary_df)

   No_Trade  Winning_Ratio  Profit_Average  Loss_Average  Payoff_Ratio  \
0       262        0.48855      942.367344   -765.653284      1.230802   

   Profit_Factor  Max_Drawdown  Portfolio_Max_Drawdown  Max_Cumulative_Profit  \
0       1.175691     54.894937               44.339627               20293.39   

   Max_Cumulative_Loss  Last_Cumulative  
0             -5238.12         18025.48  
